Final Project : MultiThreaded Web Server
---

It’s been a long journey, but we’ve reached the end of the book. In this  
chapter, we’ll build one more project together to demonstrate some of the  
concepts we covered in the final chapters, as well as recap some earlier  
lessons.

For our final project, we’ll make a web server that says “hello” and looks  
like Figure 20-1 in a web browser.

![image](https://doc.rust-lang.org/book/img/trpl20-01.png)

> Figure 20-1: Our final shared project

Here is our plan for building the web server:

1. Learn a bit about TCP and HTTP.
2. Listen for TCP connections on a socket.
3. Parse a small number of HTTP requests.
4. Create a proper HTTP response.
5. Improve the throughput of our server with a thread pool.

Before we get started, we should mention one detail:  

- the method we’ll use won’t be the best way to build a web server with Rust.  

Community members have published a number of production-ready crates available  
on crates.io that provide more complete web server and thread pool  
implementations than we’ll build. However, our intention in this chapter is to  
help you learn, not to take the easy route. Because Rust is a systems  
programming language, we can choose the level of abstraction we want to work  
with and can go to a lower level than is possible or practical in other  
languages. We’ll therefore write the basic HTTP server and thread pool manually  
so you can learn the general ideas and techniques behind the crates you might  
use in the future.

### Building a Single-Threaded Web Server

We’ll start by getting a single-threaded web server working. Before we begin,  
let’s look at a quick overview of the protocols involved in building web  
servers. The details of these protocols are beyond the scope of this book, but  
a brief overview will give you the information you need.

The two main protocols involved in web servers are Hypertext Transfer Protocol  
(HTTP) and Transmission Control Protocol (TCP). Both protocols are  
request-response protocols, meaning a client initiates requests and a server  
listens to the requests and provides a response to the client. The contents of  
those requests and responses are defined by the protocols.

TCP is the lower-level protocol that describes the details of how information  
gets from one server to another but doesn’t specify what that information is.  
HTTP builds on top of TCP by defining the contents of the requests and  
responses. It’s technically possible to use HTTP with other protocols, but in  
the vast majority of cases, HTTP sends its data over TCP. We’ll work with the  
raw bytes of TCP and HTTP requests and responses.

### Listening to the TCP Connection

Our web server needs to listen to a TCP connection, so that’s the first part  
we’ll work on. The standard library offers a `std::net` module that lets us do  
this. Let’s make a new project in the usual fashion:

```sh
$ cargo new hello
     Created binary (application) `hello` project
$ cd hello
```

Now enter the code in Listing 20-1 in src/main.rs to start. This code will  
listen at the local address 127.0.0.1:7878 for incoming TCP streams. When it  
gets an incoming stream, it will print Connection established!.

> Filename: src/main.rs

In [ ]:
use std::net::TcpListener;

{
    // binding the specific spot in our computer's memory where the listener
    // is setup
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        println!("Connection established!");
    }
}


> Listing 20-1: Listening for incoming streams and printing a message when we receive a stream

Using TcpListener, we can listen for TCP connections at the address  
127.0.0.1:7878. In the address, the section before the colon is an IP address  
representing your computer (this is the same on every computer and doesn’t  
represent the authors’ computer specifically), and 7878 is the port. We’ve  
chosen this port for two reasons:  

- HTTP isn’t normally accepted on this port so our server is unlikely to  
conflict with any other web server you might have running on your machine, and  
- 7878 is rust typed on a telephone.

The bind function in this scenario works like the new function in that it will  
return a new TcpListener instance. The function is called bind because, in  
networking, connecting to a port to listen to is known as “binding to a port.”

The bind function returns a Result<T, E>, which indicates that it’s possible  
for binding to fail. For example, connecting to port 80 requires administrator  
privileges (nonadministrators can listen only on ports higher than 1023), so if  
we tried to connect to port 80 without being an administrator, binding wouldn’t  
work. Binding also wouldn’t work, for example, if we ran two instances of our  
program and so had two programs listening to the same port. Because we’re  
writing a basic server just for learning purposes, we won’t worry about  
handling these kinds of errors; instead, we use unwrap to stop the program if  
errors happen.

The incoming method on TcpListener returns an iterator that gives us a sequence  
of streams (more specifically, streams of type TcpStream). A single stream  
represents an open connection between the client and the server. A connection  
is the name for the full request and response process in which a client  
connects to the server, the server generates a response, and the server closes  
the connection. As such, we will read from the TcpStream to see what the client  
sent and then write our response to the stream to send data back to the client.  
Overall, this for loop will process each connection in turn and produce a  
series of streams for us to handle.

For now, our handling of the stream consists of calling unwrap to terminate our  
program if the stream has any errors; if there aren’t any errors, the program  
prints a message. We’ll add more functionality for the success case in the next  
listing. The reason we might receive errors from the incoming method when a  
client connects to the server is that we’re not actually iterating over  
connections. Instead, we’re iterating over connection attempts. The connection  
might not be successful for a number of reasons, many of them operating system  
specific. For example, many operating systems have a limit to the number of  
simultaneous open connections they can support; new connection attempts beyond  
that number will produce an error until some of the open connections are closed.

Let’s try running this code! Invoke cargo run in the terminal and then  
load 127.0.0.1:7878 in a web browser. The browser should show an error message  
like “Connection reset,” because the server isn’t currently sending back any  
data. But when you look at your terminal, you should see several messages that  
were printed when the browser connected to the server!

```sh
Running `target/debug/hello`
Connection established!
Connection established!
Connection established!
```

Sometimes, you’ll see multiple messages printed for one browser request; the  
reason might be that the browser is making a request for the page as well as a  
request for other resources, like the favicon.ico icon that appears in the  
browser tab.

It could also be that the browser is trying to connect to the server multiple  
times because the server isn’t responding with any data. When stream goes out  
of scope and is dropped at the end of the loop, the connection is closed as  
part of the drop implementation. Browsers sometimes deal with closed  
connections by retrying, because the problem might be temporary. The important  
factor is that we’ve successfully gotten a handle to a TCP connection!

Remember to stop the program by pressing ctrl-c when you’re done running a  
particular version of the code. Then restart the program by invoking the cargo  
run command after you’ve made each set of code changes to make sure you’re  
running the newest code.

### Reading the Request

Let’s implement the functionality to read the request from the browser! To  
separate the concerns of first getting a connection and then taking some action  
with the connection, we’ll start a new function for processing connections. In  
this new handle_connection function, we’ll read data from the TCP stream and  
print it so we can see the data being sent from the browser. Change the code to  
look like Listing 20-2.

> Filename: src/main.rs

In [ ]:
use std::{
    io::{prelude::*, BufReader},
    net::{TcpListener, TcpStream},
}

fn handle_connection(mut stream: TcpStream){
    let buf_reader = BufReader::new(&mut stream);
    let http_request: Vec<_> = buf_reader
        .lines()
        .map(|result| result.unwrap())
        .take_while(|line| !line.is_empty())
        .collect();

    println!("Request: {:#?}", http_request);
}

{
    let listener = TcpListener::bind("127.0.0.1:7878").unwrap();

    for stream in listener.incoming() {
        let stream = stream.unwrap();
        handle_connection(stream);
    }
}